# 1. Instalación e importación de bibliotecas

In [2]:
!git clone https://github.com/angelfergar/ChurnRetention-SP.git

fatal: destination path 'ChurnRetention-SP' already exists and is not an empty directory.


In [3]:
import zipfile
import os

import pandas as pd
import joblib
import numpy as np

import xgboost as xgb
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorIndexer,VectorAssembler, StringIndexer
from pyspark.ml.classification import GBTClassificationModel
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import DoubleType

In [4]:
# Creamos la sesión de Spark
spark = SparkSession.builder.appName('Simulación').getOrCreate()

In [5]:
# Cargamos el dataset y los modelos que usaremos para la simulación
dataset_path = '/content/ChurnRetention-SP/data/preprocessed_customerChurn.csv'
df = pd.read_csv(dataset_path)

# Cargamos el modelo XGBoost
model_path = '/content/ChurnRetention-SP/models/best_model.joblib'
model = xgb.Booster()
model = joblib.load(model_path)

# Descomprimir el modelo de Spark antes de cargarlo
spark_path = '/content/ChurnRetention-SP/models/best_spark_model.zip'
extract_path = '/content/ChurnRetention-SP/models/best_spark_model/'

# Descomprimir el modelo si no está extraído
if not os.path.exists(extract_path):
    with zipfile.ZipFile(spark_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Cargamos el GBT Classifier
spark_model = GBTClassificationModel.load(extract_path)

# 2. Preparación del dataset

Tenemos que eliminar la columna Churn Label, ya que no es útil a la hora de realizar la simulación.

También tenemos que crear una columna donde asignaremos la probabilidad que tiene cada cliente de darse de baja en base al modelo entrenado que tenemos. Esto nos permitirá conocer cuantos clientes en riesgo etenemos entre nuestros clientes.

In [6]:
# Eliminamos la columna de Churn Label para evitar añadir ruido en la simulación
df.drop(columns='Churn Label', inplace=True) # ML Estándar

Una vez tenemos el dataset base listo, podemos crear el dataset que usaremos con el modelo de Spark y vectorizarlo

In [7]:
df_spark = spark.createDataFrame(df)

feature_columns = [col for col in df.columns if col != 'Churn Label']

# Ponemos las features en un solo vector para que puedan ser usadas por modelos de ML de Spark
assembler = VectorAssembler(inputCols=feature_columns, outputCol='rawFeatures')
df_spark = assembler.transform(df_spark)

feature_indexer = VectorIndexer(inputCol='rawFeatures', outputCol='features')
df_spark = feature_indexer.fit(df_spark).transform(df_spark)

Una vez tenemos los datasets listos, calculamos la probabilidad de cada cliente individual de darse de baja

In [8]:
# Predecimos la probabilidad de churn y la mostramos en una nueva columna
X_actual = df
df['Churn_Probability'] = model.predict_proba(X_actual)[:, 1]

# Hacemos lo mismo en Spark
extract_prob = udf(lambda prob: float(prob[1]), DoubleType())

# Generamos las predicciones para cada cliente
predictions = spark_model.transform(df_spark)
df_pred = predictions.select(extract_prob(col('probability')).alias('Churn_Probability'))

Con la probabilidad calculada, contamos todos aquellos clientes que hayan obtenido por encima de un 60% de probabilidad de darse de baja. Estos serán nuestros clientes en riesgo

In [9]:
# Definimos el umbral a partir del cual consideramos como un Cliente en riesgo
df_riesgo = df[df['Churn_Probability'] > 0.6]
print(f'Clientes en riesgo detectados con el modelo de XGBoost: {len(df_riesgo)}')

# Hacemos los mismo con Spark
df_riesgo_spark = df_pred.filter(col("Churn_Probability") > 0.6)
print(f'Clientes en riesgo detectados con el modelo GBT Classifier (Spark): {df_riesgo_spark.count()}')

Clientes en riesgo detectados con el modelo de XGBoost: 1742
Clientes en riesgo detectados con el modelo GBT Classifier (Spark): 2067


Obtemos que de los 7043 clientes que aparecen en el dataset:

*   El modelo XGBoost predice que hay 1742 clientes en riesgo de darse de baja
*   El modelo GBT Classifier de Spark predice que hay 2067 clientes en riesgo de darse de baja


# 3. Simulación

En esta simulación tenemos como objetivo conocer cómo impactaría una mejora que supusiese un incremento en la retención de clientes. Para ello obtendremos:


1.   Los clientes potenciales que retenemos una vez se aplica la mejora
2.   El ahorro que supone para la empresa esta retención de usuarios



In [10]:
# Parámetros de simulación
total_clientes = len(df) #Usamos esta variable para ambas simulaciones, ya que es el mismo número
tasa_churn_actual = 0.265  # Churn base (26.5%) - Contamos con este dato de antemano
clientes_perdidos_actual = int(total_clientes * tasa_churn_actual)

Establecemos el contexto financiero con el que trabajamos

In [11]:
# Supuestos financieros
ingreso_mensual_promedio = 50  # Supongamos que cada cliente aporta 50€/mes
costo_adquisicion_cliente = 300  # El supuesto coste de adquirir un nuevo cliente es de 300€

Calculamos la situacióne económica actual sin haber aplicado ninguna estrategia de retención de clientes

In [12]:
# Escenario sin intervención
perdida_actual = clientes_perdidos_actual * ingreso_mensual_promedio * 12
coste_reemplazo_clientes = clientes_perdidos_actual * costo_adquisicion_cliente
perdida_total_actual = perdida_actual + coste_reemplazo_clientes

In [13]:
print(f'Clientes perdidos sin aplicar ninguna estrategia: {clientes_perdidos_actual}')
print(f'Pérdida total sin intervención: {perdida_total_actual:,.2f}€') # La cantidad que perdemos sin aplicar ninguna estrategia

Clientes perdidos sin aplicar ninguna estrategia: 1866
Pérdida total sin intervención: 1,679,400.00€


## 3.1 Simulación modelo estándar

In [14]:
# Impacto de estrategias de retención
retencion_estrategia = 0.35  # Supongamos que retenemos el 55% con una mejora de la Fibra Óptica

# Obtenemos el número de clientes que mantendríamos con estas mejoras
clientes_rescatados = int(len(df_riesgo) * retencion_estrategia)

In [15]:
# Suponemos que la mejora de la fibra no tiene coste extra significativo y obtenemos el ahorro que nos da esta mejora
ahorro_fibra = clientes_rescatados * ingreso_mensual_promedio * 12

In [16]:
# Mostramos los beneficios de la aplicación de la estrategia de retención
print(f'Clientes en riesgo retenidos con estrategia (Fibra Óptica): {clientes_rescatados}')
print(f'Ahorro con estrategia de fibra óptica: {ahorro_fibra:,.2f}€')

Clientes en riesgo retenidos con estrategia (Fibra Óptica): 609
Ahorro con estrategia de fibra óptica: 365,400.00€


## 3.2 Simulación modelo Spark

In [17]:
# Impacto de estrategias de retención
retencion_estrategia_s = 0.35  # Supongamos que retenemos el 55% con una mejora de la Fibra Óptica

# Obtenemos el número de clientes que mantendríamos con estas mejoras
clientes_rescatados_s = int((df_riesgo_spark.count()) * retencion_estrategia)

In [18]:
# Suponemos que la mejora de la fibra no tiene coste extra significativo y obtenemos el ahorro que nos da esta mejora
ahorro_fibra_s = clientes_rescatados_s * ingreso_mensual_promedio * 12

In [19]:
# Mostramos los clientes que se podrían retener con la mejora
print(f'Clientes en riesgo retenidos con estrategia (Fibra Óptica): {clientes_rescatados_s}')
print(f'Ahorro con estrategia de fibra óptica: {ahorro_fibra_s:,.2f}€')

Clientes en riesgo retenidos con estrategia (Fibra Óptica): 723
Ahorro con estrategia de fibra óptica: 433,800.00€


## Resultados finales

Inicialmente, el contexto en el que se trataba era:


*   1866 clientes perdidos
*   Pérdidas 1,679,400.00€ debido a la cantidad de clientes perdidos y el coste de adquisición de nuevos clientes



Utilizando los modelo de machine learning que se han entrenado (Estándar y Spark), se simula una situación en la que se **incrementa la retención de los clientes un 35% al mejorar el servicio de fibra óptica** (Uno de los puntos clave a la hora de que se den de baja los clientes).

Los resultados de esta mejora según el modelo de machine learning estándar se traducen en:

*   **609 clientes retenidos**
*   **Un ahorro de 365,400.00€** gracias a la retención de clientes

Por su parte, el modelo de Spark entrenado muestra que:

*   **Se retienen 723 clientes**
*   La retención de clientes se transforma en un **ahorro de 433,800.00€**






Este sistema se puede aplicar con otras estrategias de retención, siendo modular y fácilmente modificable